In [3]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import random
import scipy
import faker


In [4]:
from copy import deepcopy
fake= faker.Faker()
accounts=range(1,20)
col_list=['id','line','account','date_trans','date_entry','dc','amount','seed']
def create_je(id):
    je_date= fake.date_between(start_date='-1y', end_date='today')
    je_delay= min(je_date+timedelta(days=5),datetime.date(datetime.now()))
    je_date_entry=fake.date_between_dates(date_start=je_date  , date_end=je_delay)
    je=[]
    deb=0
    m=random.randint(1,4)
    acum=0
    je_line=0
    accounts_sample= random.sample(accounts, 8)
    for n in range(m):
        je_amount=round(random.random()*50000,2)
        if je_amount>0:
            je_line=je_line+1
            je_account=accounts_sample[je_line-1]
            je.append([id,je_line,je_account, je_date,je_date_entry,"D", je_amount,0])
            acum=acum+je_amount
        
    m=random.randint(1,4)
    for n in range(m):
        je_amount=round(random.random()*50000,2)
        if acum-je_amount<0:
            je_amount=round(acum,2)
        if je_amount>0:
            je_line=je_line+1
            je_account=accounts_sample[je_line-1]
            je.append([id,je_line,je_account,je_date,je_date_entry,"C", -je_amount,0])
            acum=acum-je_amount
            if acum==0:
                break
    return je



trans=[]
counter=1
stop=1000
while counter<stop:    
    je=create_je(counter)
    for i, l in enumerate(je):
        if l[2]==3:
            je2=deepcopy(je)
            counter=counter+1
            new_date=min(je2[0][3]+timedelta(days=random.randint(0,60)),datetime.date(datetime.now()))
            new_entry_date=min(je2[0][4]+timedelta(days=random.randint(0,120)),datetime.date(datetime.now()))
            for l2 in je2:
                l2[0]=counter
                l2[3]=new_date
                l2[4]=new_entry_date
                l2[5]="D" if l2[5]=="C" else "C"
                l2[6]=-l2[6]
                l2[7]=1
            seed_type=random.choices([1,2,3],[.9,.05,.05])[0]
            if seed_type==1:
                #no seeding
                trans.extend(je2)
            if seed_type==2:
                spl=random.random()
                jel=deepcopy(je2[i])
                a=je2[i][6]
                je2.append(jel)
                je2[i][6]=round(a * spl,2)
                je2[-1][6]=round((1.0-spl) * a,2)
                je2[-1][1]=len(je2)
                je2[i][7]=seed_type
                je2[-1][7]=seed_type
                trans.extend(je2)
            if seed_type==3: 
                if len(je2)>2:
                    debits=[r[6] for r in je2 if r[5]=="D"]
                    s1=deepcopy(je2[0])
                    s2=deepcopy(je2[1])
                    s1[7]=seed_type
                    s2[7]=seed_type
                    s1[6]=sum(debits)
                    s2[6]=-sum(debits)
                    s1[5]="D"
                    s2[5]="C"
                    if random.random()>.5:
                        s1[2]=3
                        s2[2]=random.choice(accounts)
                    else:
                        s2[2]=3
                        s1[2]=random.choice(accounts)
                    je3=[s1,s2]  
                    trans.extend(je3)
                    print("joining line at id ",s1[0])
            break
    trans.extend(je)
    counter=counter+1
df=pd.DataFrame(trans, columns=col_list)

joining line at id  168
joining line at id  205
joining line at id  387
joining line at id  525
joining line at id  625
joining line at id  721
joining line at id  831
joining line at id  915
joining line at id  972
joining line at id  980


In [5]:
dfs=df.sort_values(["date_entry","date_trans",'id','dc','line'],axis=0)

In [6]:
n=0
prev=0
reindexed_list=[]
for r in dfs.itertuples():  
    if r[1] != prev:
        prev=r[1]
        n=n+1
        c=1
        new=[n,c,r[3],r[4],r[5],r[6],r[7],r[8]]
        
    else:
        c=c+1
        new=[n,c,r[3],r[4],r[5],r[6],r[7],r[8]]
    reindexed_list.append(new)
        
dfn= pd.DataFrame(reindexed_list,columns=col_list)

In [7]:
dfn[dfn.seed==3]

,id,line,account,date_trans,date_entry,dc,amount,seed
837,181,1,14,2020-11-10,2020-09-16,C,-91882.55,3
838,181,2,3,2020-11-10,2020-09-16,D,91882.55,3
1381,292,1,11,2020-08-14,2020-10-26,C,-50789.73,3
1382,292,2,3,2020-08-14,2020-10-26,D,50789.73,3
1818,383,1,3,2020-11-28,2020-11-26,C,-35356.19,3
1819,383,2,16,2020-11-28,2020-11-26,D,35356.19,3
2700,566,1,3,2021-01-30,2021-01-31,C,-40994.89,3
2701,566,2,9,2021-01-30,2021-01-31,D,40994.89,3
2929,612,1,16,2020-11-16,2021-02-21,C,-23992.01,3
2930,612,2,3,2020-11-16,2021-02-21,D,23992.01,3


In [10]:
dfn.to_excel("data.xlsx",index=False)

In [10]:
df[df.id==5]

,id,line,account,date_trans,date_entry,dc,amount,seed
13,5,1,16,2021-02-09,2021-02-10,D,6595.0,0
14,5,2,15,2021-02-09,2021-02-10,C,-3159.0,0


In [5]:
random.random()

0.20932934354744648